In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nbdev import *
%nbdev_default_export physics
%nbdev_default_class_level 3

Cells will be exported to geomechy.physics,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [3]:
from geomechy.core import *

In [4]:
%nbdev_export
class flux:
    pass

In [5]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted index.ipynb.
